In [3]:
import requests
import geopandas as gpd
import pandas as pd

In [46]:
from elasticsearch import Elasticsearch

ec = Elasticsearch(hosts=[f"http://localhost:9300"])

body = {
    "mappings": {
        "properties": {
            "body_vector": {
                "type": "dense_vector",
                "dims": 4096,
                "index": True,
                "similarity": "cosine",
            },
            "body": {"type": "text"},
            "num_id": {"type": "long"},
            "object_id": {"type": "keyword"},
            "location": {"type": "geo_shape"},
            "properties": {
                "type": "object",
                "enabled": True
            }
        }
    }
}

ec.indices.create(
    index="1830&analyze",
    body=body
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': '1830&analyze'})

In [11]:
normative_df = pd.read_excel("ЗОУИТ.xlsx", sheet_name="Загружено в базу")
normative_df = normative_df.dropna(subset="ЗОУИТ").drop(columns=["№", "buffer_type_id", "Буфер (в метрах)", "ID сервиса или физика"])
normative_df.to_parquet("normative_data_cleaned.parquet")

In [12]:
normative_df = pd.read_parquet("normative_data_cleaned.parquet")

In [13]:
normative_df

,ЗОУИТ,Ограничения (строгие),Физический объект,Связанные объекты/сервисы из ограничений
0,"Зоны затопления, подтопления","В границах зон затопления, подтопления запреща...",Река,"1) Объекты капитального строительства, не обес..."
5,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,ТБО,"1) Жилая застройка, включая отдельные жилые до..."
9,Придорожная полоса автомобильных дорог,В границах полосы отвода автомобильной дороги ...,Федеральная дорога,"1) здания, строения, сооружения и другие объек..."
12,Охранные зоны объектов электросетевого хозяйства,В охранных зонах запрещается осуществлять любы...,ЛЭП,"1) Свалка\n 2) хранилища любых, в том числе го..."
14,Охранные зоны тепловых сетей,В пределах охранных зон тепловых сетей не допу...,Объект теплоснабжения,1)автозаправочные станции\n 2) хранилища горюч...
16,Охранные зоны газораспределительных сетей,"На земельные участки, входящие в охранные зоны...",Распределительный трубопровод для транспортиро...,1) объекты жилищно-гражданского и производстве...
18,Охранные зоны магистральных трубопроводов,"Земельные участки, входящие в охранные зоны тр...",Объект магистрального газопровода,1) свалки\n 2) берегоукрепительные сооружения...
19,Охранные зоны линий и сооружений связи и линий...,На трассах радиорелейных линий связи в целях п...,Сеть электросвязи,"1) всякого рода строительные, монтажные и взры..."


In [19]:
all_buffers_resp = requests.get(
    "/api/v1/scenarios/1830/buffers",
    headers=headers
)

from shapely.geometry import shape

df = pd.json_normalize(all_buffers_resp.json()["features"], sep="_")
df["geometry"] = df.apply(lambda x: shape({"type": x["geometry_type"], "coordinates": x["geometry_coordinates"]}), axis=1)
gdf = gpd.GeoDataFrame(df.drop(columns=["geometry_type", "geometry_coordinates", "type", "properties_is_scenario_object", "properties_urban_object_object_geometry_territory_name", "properties_urban_object_object_geometry_territory_id"]).rename(columns={"properties_urban_object_object_geometry_id": "geometry_id", "properties_urban_object_physical_object_type_name": "physical_object_type", "properties_urban_object_physical_object_type_id": "object_type_id", "properties_urban_object_physical_object_name": "physical_object_name", "properties_buffer_type_description": "description", "properties_buffer_type_name": "buffer_type-name"}), geometry="geometry", crs=4326)

In [29]:
from shapely import union_all

zones_gdf = gpd.GeoDataFrame(gdf.groupby("physical_object_type").agg(
    {
        "buffer_type-name": "first",
        "description": "first",
        "geometry": union_all
    }
).reset_index(), geometry="geometry", crs=4326)
zones_gdf.explore()

In [30]:
zones_gdf = zones_gdf.merge(normative_df, how="left", left_on="buffer_type-name", right_on="ЗОУИТ")

In [31]:
zones_gdf.to_parquet("zones_gdf.parquet")

In [52]:
list_to_concat = []
for i in (4, 5):
    current_objects = requests.get(
        "/api/v1/scenarios/1830/geometries_with_all_objects",
        params={"physical_object_type_id": i},
        headers=headers
    )
    current_objects_gdf = gpd.GeoDataFrame.from_features(current_objects.json(), crs=4326)
    current_objects_gdf.drop(columns=["territory", "object_geometry_id", "osm_id"], inplace=True)
    list_to_concat.append(current_objects_gdf)

all_objects_gdf = pd.concat(list_to_concat, ignore_index=True)

In [53]:
all_objects_gdf.sjoin(zones_gdf, how="left").drop(columns=["is_scenario_object", "is_locked", "index_right", "ЗОУИТ", "description", "Физический объект"]).rename(columns={"Связанные объекты/сервисы из ограничений": "На какие объекты влияет зона"})

,geometry,address,physical_objects,services,physical_object_type,buffer_type-name,Ограничения (строгие),На какие объекты влияет зона
0,"POLYGON ((30.725 60.04121, 30.72533 60.0414, 3...",None,"[{'physical_object_id': 1474596, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
1,"POLYGON ((30.72542 60.04308, 30.72517 60.0431,...",None,"[{'physical_object_id': 1474601, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
2,"POLYGON ((30.72483 60.04255, 30.7247 60.04264,...",None,"[{'physical_object_id': 1474606, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
3,"POLYGON ((30.73406 60.05145, 30.73397 60.05153...",None,"[{'physical_object_id': 1474583, 'physical_obj...","[{'service_id': 117290, 'service_type': {'id':...",Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
4,"POLYGON ((30.72343 60.04831, 30.72346 60.04833...",None,"[{'physical_object_id': 1474611, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
...,...,...,...,...,...,...,...,...
362,"POLYGON ((30.72876 60.04294, 30.72881 60.04293...",None,"[{'physical_object_id': 1474925, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
363,"POLYGON ((30.72794 60.04437, 30.72801 60.04427...",None,"[{'physical_object_id': 1474931, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
364,"POLYGON ((30.72482 60.04664, 30.72591 60.047, ...",None,"[{'physical_object_id': 1474937, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."
365,"POLYGON ((30.72104 60.0377, 30.72065 60.03787,...",None,"[{'physical_object_id': 1474943, 'physical_obj...",[],Промышленная территория,"Санитарно-защитные зоны (СЗЗ), устанавливаемые...",В санитарно-защитной зоне запрещается размещен...,"1) Жилая застройка, включая отдельные жилые до..."


In [60]:
from shapely import to_geojson

all_objects_gdf["properties"] = all_objects_gdf.drop(columns="geometry").apply(lambda x: x.to_dict())
all_objects_gdf["location"] = all_objects_gdf["geometry"].apply(to_geojson)
all_objects_gdf["object_id"] = all_objects_gdf["physical_objects"].apply(lambda x: x[0]["physical_object_id"])

In [61]:
data_do_upload = all_objects_gdf.drop(columns="geometry").to_dict(orient="records")

In [70]:
index_data = []
for i in data_do_upload:
    row = {
        "text": str(
            {k: v for k, v in i.items() if k not in ("geometry", "physical_objects", "location", "properties")}
        ),
        "object_id": str(i["object_id"]),
        "location": i["location"],
        "properties": i["properties"]
    }
    index_data.append(row)

In [74]:
index_data[0]["object_id"]

'1474596'

In [72]:
import json

with open("index_data.json", "w") as f:
    json.dump(index_data, f)